In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# inte
miss_df = pd.DataFrame({'inv_bool': [1,0,1,1,0,0,1,1,1,0,0,0,1,1,1,1,0,0,0,0,1,1,1,1,1,0,0,0,0,0,1,1,1,1,1,1,0,0,0,0,0]})

miss_df['inv_bool'] = miss_df['inv_bool'].replace(0, np.nan)

# get local cumsum of 'inv_bool' column:
miss_df['cumsum'] = miss_df['inv_bool'].isnull().astype(int).groupby(miss_df['inv_bool'].notnull().astype(int).cumsum()).cumsum()

# create col 'plus' with values from 'cumsum' added by 0.1 skiping 0s:
miss_df['plus'] = miss_df['cumsum'].where(miss_df['cumsum'] != 0, np.NaN) + 0.1

# fill NaN with 0s:
miss_df['plus'] = miss_df['plus'].fillna(0)

# get diff of 'plus' column shifted one row up:
miss_df['diff_up'] = miss_df['plus'].diff().shift(-1)

# create col 'end' with index value if .lt(-1.1):
miss_df['end'] = miss_df[miss_df['diff_up'].lt(-1.1)]['diff_up']

# fill NaN with 0s for col 'end':
miss_df['end'] = miss_df['end'].fillna(0)

# create 'index' column as first column with index values:
try:
    miss_df.insert(0, 'index', miss_df.index)
except:
    pass

# create col 'end_index' that use .ne() for NaNs in column 'end' and return index values from 'index' column:
miss_df['end_index'] = miss_df[miss_df['end'].ne(0)]['index']

# create col 'diff' with diff for 'plus' column:
miss_df['diff'] = miss_df['plus'].diff()

# create col 'start_index' that use .eq() for 1.1 in column 'diff' and return index values from 'index' column added by 1:
miss_df['start_index'] = miss_df[miss_df['diff'].eq(1.1)]['index']

# copy 'start_index' to as df 'start_df':
start_df = miss_df['start_index'].copy()

# delete NaNs from 'start_df':
start_df = start_df.dropna()

# reset index for 'start_df':
start_df = start_df.reset_index(drop=True)

# copy 'end_index' to as df 'end_df':
end_df = miss_df['end_index'].copy()

# delete NaNs from 'end_df':
end_df = end_df.dropna()

# reset index for 'end_df':
end_df = end_df.reset_index(drop=True)

# create df 'missing_data_index_count' with cols 'start_df' and 'end_df':
missing_data_index_count = pd.DataFrame()

missing_data_index_count['start_df'] = start_df
missing_data_index_count['end_df'] = end_df

# create col 'missing_count' with diff of 'end_df' and 'start_df' columns plus 1:
missing_data_index_count['missing_count'] = missing_data_index_count['end_df'] - missing_data_index_count['start_df'] + 1

# check:
print(missing_data_index_count)
